# GRPO Fine-tuning for verifiable tasks

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/huggingface/trl/blob/main/examples/notebooks/grpo_trl_lora_qlora.ipynb)

This tutorial demonstrates how to fine-tune LFM2.5 models using GRPO (Group Relative Policy Optimization) with the TRL library for tasks with verifiable outcomes.

Follow along if it's your first time using GRPO, or take single code snippets for your own workflow

### 🎯 What You'll Find:
- **GRPO Training** - Optimizing model outputs using outcome-based rewards
- **Reward Modeling** - Setting up verification functions for correct answers
- **LoRA + GRPO** - (Optional) using LoRA (from PEFT) to train efficiently on constrained hardware

### 📋 Prerequisites:
- **GPU Runtime**: Select GPU in `Runtime` → `Change runtime type`
- **Hugging Face Account**: For accessing models and datasets
- **Verifiable Dataset**: Tasks with clear correct/incorrect outcomes (e.g., math problems, code execution, structured data extraction, translation with reference answers)

### 💡 Example Use Cases:
- Mathematical problem solving with numeric verification
- Code generation with unit test validation
- Structured output tasks (JSON, SQL) with schema validation
- Question answering with ground truth answers
```

## 📦 Installation & Setup

First, let's install all the required packages:

We'll install **TRL** with the **PEFT** extra, which ensures all main dependencies such as **Transformers** and **PEFT** (a package for parameter-efficient fine-tuning, e.g., LoRA/QLoRA) are included.

Additionally, we'll install
- **trackio** to log and monitor our experiments
- **bitsandbytes** to enable quantization of LLMs, reducing memory consumption for both inference and training, and
- **liger-kernel** for more efficient training.

In [1]:
!pip install -Uq "trl[peft]" bitsandbytes trackio math_verify liger-kernel
!pip install flash-attn --no-build-isolation

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.1/209.1 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.6/230.6 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.3/24.3 MB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.9/532.9 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 93.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for flash-attn: filename=flash_attn-2.8.3-cp312-cp312-linux_x86_64.whl size=253780426 sha256=4e2f9e39313266b1544b68138b15b91ee6221eccf14f7902b7c6620351340810
  Stored in directory: /root/.cache/pip/wheels/3d/59/46/f282c12c73dd4bb3c2

In [ ]:
import torch
import transformers
import trl
import os
os.environ["WANDB_DISABLED"] = "true"

print(f"📦 PyTorch version: {torch.__version__}")
print(f"🤗 Transformers version: {transformers.__version__}")
print(f"📊 TRL version: {trl.__version__}")

## Load the dataset

In this step, we load the [**AI-MO/NuminaMath-TIR**](https://huggingface.co/datasets/AI-MO/NuminaMath-TIR) dataset from the Hugging Face Hub using the `datasets` library.
This dataset focuses on **mathematical reasoning**, featuring problems that require step-by-step logical solutions.
By fine-tuning a model that does not yet exhibit strong reasoning capabilities, it can learn to **generate structured reasoning steps**, enhancing both the model's **accuracy** and **interpretability** on math-related tasks.

For efficiency, we'll load only a **small portion of the training split**:

In [ ]:
from datasets import load_dataset

dataset_name = 'AI-MO/NuminaMath-TIR'
train_dataset = load_dataset(dataset_name, split='train[:5%]')

# Check the structure of the dataset
print(train_dataset)
print(train_dataset[0])

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/147M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/215k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/72441 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/99 [00:00<?, ? examples/s]

## Transform the dataset

We will adapt our dataset to a conversational format using a custom system prompt, guiding the LLM to generate both step-by-step reasoning and the final answer.

In [ ]:
# Transform
SYSTEM_PROMPT = (
    "A conversation between User and Assistant. The user asks a question, and the Assistant solves it. The assistant  "
    "first thinks about the reasoning process in the mind and then provides the user with the answer. The reasoning "
    "process is enclosed strictly within <think> and </think> tags. "
    "After closing </think>, the assistant MUST provide the final answer in plain text."
)


def make_conversation(example):
    return {
        "prompt": [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": example["problem"]},
        ],
    }

train_dataset = train_dataset.map(make_conversation)

# remove unused columns
train_dataset = train_dataset.remove_columns(['messages', 'problem'])

# Check the structure of the dataset
print(train_dataset)
print(train_dataset[0]['prompt'])

Map:   0%|          | 0/3622 [00:00<?, ? examples/s]

## Load the model

This notebook can be used with two fine-tuning methods. By default, it is set up for **QLoRA**, which includes quantization using `BitsAndBytesConfig`. If you prefer to use standard **LoRA** without quantization, simply comment out the `BitsAndBytesConfig` configuration (training without quantization consumes more memory).

In [ ]:
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig

model_id, output_dir = "LiquidAI/LFM2.5-1.2B-Instruct", "LFM2.5-1.2B-Instruct-GRPO"

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    # attn_implementation="flash_attention_2",                   # Change to Flash Attention if GPU has support
    dtype="bfloat16",                          # Change to bfloat16 if GPU has support
    # quantization_config=BitsAndBytesConfig(
    #     load_in_4bit=True,                        # Load the model in 4-bit precision to save memory
    #     bnb_4bit_compute_dtype=torch.float16,     # Data type used for internal computations in quantization
    #     bnb_4bit_use_double_quant=True,           # Use double quantization to improve accuracy
    #     bnb_4bit_quant_type="nf4"                 # Type of quantization. "nf4" is recommended for recent LLMs
    # )
)

## Define LoRA adapters

The following cell defines LoRA (or QLoRA if needed). When training with LoRA/QLoRA, we use a **base model** (the one selected above) and, instead of modifying its original weights, we fine-tune a **LoRA adapter**, a lightweight layer that enables efficient and memory-friendly training. The **`target_modules`** specify which parts of the model (e.g., attention or projection layers) will be adapted by LoRA during fine-tuning.

In [ ]:
from peft import LoraConfig
peft_config = LoraConfig(
    r=32,
    lora_alpha=32,
    target_modules = ["q_proj", "k_proj", "v_proj", "out_proj", "in_proj", "w1", "w2", "w3"],
)

## Load reward functions or define your own ones

GRPO requires **reward functions** to guide the learning process. For convenience, we can directly load pre-defined rewards from `trl.rewards`, which already includes a [collection of ready-to-use rewards](https://huggingface.co/docs/trl/rewards).

If you want to create your own custom reward functions to teach the model, a reward function is simply a Python function that takes the generated completions and returns a list of floats. For example, the following function, which we use in this notebook, rewards completions that correctly follow the `<think>` format:

```python
def think_format_reward(completions: list[list[dict[str, str]]], **kwargs) -> list[float]:
    pattern = r"^<think>(?!.*<think>)(.*?)</think>.*$"
    completion_contents = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, content, re.DOTALL | re.MULTILINE) for content in completion_contents]
    return [1.0 if match else 0.0 for match in matches]
```

In this notebook, we will use both `think_format_reward`, which rewards completions that correctly follow the `<think>` format, and `reasoning_accuracy_reward`, which evaluates the correctness of the model's solution to the mathematical problem. Together, these rewards guide the model to generate **structured reasoning** while producing **accurate answers**.

In [ ]:
from trl.rewards import think_format_reward, reasoning_accuracy_reward

## Train model

In [ ]:
from trl import GRPOConfig

# Configure training arguments using GRPOConfig
training_args = GRPOConfig(
    # Training schedule / optimization
    learning_rate=2e-5,                                     # Learning rate for the optimizer
    #num_train_epochs=1,
    max_steps=30,                                          # Number of dataset passes. For full trainings, use `num_train_epochs` instead

    # Parameters that control GRPO training (you can adapt them)
    per_device_train_batch_size = 8,
    max_completion_length=256, # default: 256               # Max completion length produced during training
    num_generations=8, # default: 8                         # Number of generations produced during trainig for comparison

    # Optimizations
    optim = "paged_adamw_8bit",                             # Optimizer
    use_liger_kernel=True,                                  # Enable Liger kernel optimizations for faster training
    gradient_checkpointing=True,                            # Save memory by re-computing activations during backpropagation

    # Parameters related to reporting and saving
    output_dir=output_dir,                                  # Where to save model checkpoints and logs
    logging_steps=10,                                       # Log training metrics every N steps
    # report_to="trackio",                                    # Experiment tracking tool
    # trackio_space_id=output_dir,                            # HF Space where the experiment tracking will be saved
    log_completions=False,                                  # Return model completions during training

    # Hub integration
    # push_to_hub=True,                                       # Automatically push the trained model to the Hugging Face Hub
                                                            # The model will be saved under your Hub account in the repository named `output_dir`
    # vLLM params
    #use_vllm=False,                                        # Activate vLLM training for faster training
    #vllm_mode='colocate',
    #vllm_gpu_memory_utilization=0.1,
    #vllm_enable_sleep_mode=True
)

from trl import GRPOTrainer
trainer = GRPOTrainer(
    model=model,
    reward_funcs=[think_format_reward, reasoning_accuracy_reward],
    args=training_args,
    train_dataset=train_dataset,
    peft_config=peft_config,
)

trainer_stats = trainer.train()

## (Optional) Save fine tuned model

In this step, we save the fine-tuned model **locally**.

In [ ]:
trainer.save_model(output_dir)